In [1]:
#Q1

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import time
import json


driver = webdriver.Chrome()


driver.get('https://www.imdb.com/list/ls056092300/')


last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    
    time.sleep(2)
    
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


soup = BeautifulSoup(driver.page_source, 'html.parser')


driver.quit()


script_tag = soup.find('script', type='application/ld+json')

ratings = {}
if script_tag:
    json_data = json.loads(script_tag.string)
    for item in json_data.get('itemListElement', []):
        movie = item.get('item', {})
        title = movie.get('name', 'N/A').strip()
        rating = movie.get('aggregateRating', {}).get('ratingValue', 'N/A')
        ratings[title] = rating


movie_containers = soup.find_all('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 bnSrml dli-title')

movies = []

for container in movie_containers:
    
    raw_title = container.find('h3', class_='ipc-title__text').text.strip()
    title = ' '.join(raw_title.split(' ', 1)[1:]).strip()  

    
    metadata_container = container.find_next_sibling('div', class_='sc-b189961a-7 btCcOY dli-title-metadata')
    year = 'N/A'
    if metadata_container:
        metadata_items = metadata_container.find_all('span', class_='sc-b189961a-8 hCbzGp dli-title-metadata-item')
        if len(metadata_items) > 0:
            year = metadata_items[0].text.strip()

    
    normalized_title = title.strip()

    
    rating = ratings.get(normalized_title, 'N/A')

    movies.append([title, rating, year])

movies_df = pd.DataFrame(movies, columns=['Title', 'Rating', 'Year'])
print(movies_df.head(100))
movies_df.to_csv('top_100_indian_movies_list.csv', index=False)
print("Data has been saved to 'top_100_indian_movies_list.csv'")


                                Title  Rating  Year
0                     Ship of Theseus     8.0  2012
1                              Iruvar     8.4  1997
2                     Kaagaz Ke Phool     7.8  1959
3   Lagaan: Once Upon a Time in India     8.1  2001
4                     Pather Panchali     8.2  1955
5                           Charulata     8.1  1964
6                     Rang De Basanti     8.1  2006
7                               Dev.D     7.9  2009
8                            3 Idiots     8.4  2009
9                              Awaara     7.8  1951
10                            Nayakan     8.7  1987
11                          Aparajito     8.2  1956
12                    Pushpaka Vimana     8.6  1987
13                             Pyaasa     8.3  1957
14                      Ghatashraddha     7.5  1977
15                             Sholay     8.1  1975
16                           Aradhana     7.6  1969
17              Do Ankhen Barah Haath     8.4  1957
18          

In [2]:
#Q2

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
def scrape_patreon():
    url = 'https://www.patreon.com/coreyms'
    
    
    driver = webdriver.Chrome()
    
    driver.get(url)
    
   
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    
    page_source = driver.page_source
    
    
    soup = BeautifulSoup(page_source, 'html.parser')
    
    posts = []
    post_containers = soup.find_all('div', class_='sc-xg7a7d-0 jJOVvL')
    
    for post in post_containers:
        heading = post.find('div', class_='sc-bBHxTw jIEOUn').get_text(strip=True) if post.find('div', class_='sc-bBHxTw jIEOUn') else 'N/A'
        
        
        date = post.find('div', class_='sc-lgu5zg-0 gDgFXW').get_text(strip=True).replace('New', '').strip() if post.find('div', class_='sc-lgu5zg-0 gDgFXW') else 'N/A'
        content = post.find('div', class_='sc-cfnzm4-0 daxSFj').get_text(strip=True) if post.find('div', class_='sc-cfnzm4-0 daxSFj') else 'N/A'
        
        youtube_url = None
        video_iframe = post.find('iframe')
        if video_iframe and 'youtube.com' in video_iframe['src']:
            youtube_url = video_iframe['src']
            youtube_url = re.sub(r'&amp;.*$', '', youtube_url)  

        likes = 'N/A'
        if youtube_url:
            likes = get_youtube_likes(driver, youtube_url) 
        
        posts.append([heading, date, content, likes])
    
   
    posts_df = pd.DataFrame(posts, columns=['Heading', 'Date', 'Content', 'Likes'])
    
   
    print(posts_df.head())
    
    
    posts_df.to_csv('patreon_posts.csv', index=False)
    print("Data has been saved to 'patreon_posts.csv'")
    
    
    driver.quit()

if __name__ == "__main__":
    scrape_patreon()


                                             Heading          Date  \
0  Python Tkinter Tutorial (Part 1): Getting Star...       July 28   
1  Automating My Bill Payments with Python and Se...        June 1   
2  Python Tutorial: Securely Manage Passwords and...      April 30   
3  Automate Your Development Environment Setup wi...      April 30   
4  How to Use ChatGPT as a Powerful Tool for Prog...  May 21, 2023   

                                             Content Likes  
0                                                      N/A  
1                                                      N/A  
2  In this Python Programming video, we will be l...   N/A  
3  In this video, we'll be learning how I set up ...   N/A  
4                                                      N/A  
Data has been saved to 'patreon_posts.csv'


In [ ]:
#Q3 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://www.nobroker.in/property/sale/bangalore/multiple?searchParam=W3sibGF0IjoxMi45NzgzNjkyLCJsb24iOjc3LjY0MDgzNTYsInBsYWNlSWQiOiJDaElKa1FOM0dLUVdyanNSTmhCUUpyaEdEN1UiLCJwbGFjZU5hbWUiOiJJbmRpcmFuYWdhciJ9LHsibGF0IjoxMi45MzA3NzM1LCJsb24iOjc3LjU4MzgzMDIsInBsYWNlSWQiOiJDaElKMmRkbFo1Z1ZyanNSaDFCT0FhZi1vcnMiLCJwbGFjZU5hbWUiOiJKYXlhbmFnYXIifSx7ImxhdCI6MTIuOTk4MTczMiwibG9uIjo3Ny41NTMwNDQ1OTk5OTk5OSwicGxhY2VJZCI6IkNoSUp4Zlc0RFBNOXJqc1JLc05URy01cF9RUSIsInBsYWNlTmFtZSI6IlJhamFqaW5hZ2FyIn1d&radius=2.0&city=bangalore&locality=Indiranagar,Jayanagar,Rajajinagar'  # Example URL


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}


def scrape_houses(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    

    listings = soup.find_all('div', class_='infinite-scroll-component__outerdiv')  

    houses = []
    
    for listing in listings:
        try:
            
            title = listing.find('h2', class_='heading-6 flex items-center font-semi-bold m-0').text.strip()

            location = listing.find('div', class_='mt-0.5p overflow-hidden overflow-ellipsis whitespace-nowrap max-w-70 text-gray-light leading-4 po:mb-0.1p po:max-w-95').text.strip()

            area = listing.find('div', class_='flex flex-col w-33pe items-center tp:w-half po:w-full').text.strip()

            emi = listing.find('div', class_='flex flex-col w-33pe items-center bo tp:w-half po:w-full border-r border-r-solid border-card-overview-border-color last:border-r-1').text.strip()

            price = listing.find('div', class_='font-semi-bold heading-6').text.strip()

            
            houses.append({
                'Title': title,
                'Location': location,
                'Area': area,
                'EMI': emi,
                'Price': price
            })
        except AttributeError:
            
            continue
    
    return houses


houses = scrape_houses(url)
houses_df = pd.DataFrame(houses)
print(houses_df)
houses_df.to_csv('nobroker_house_details.csv', index=False)
print("Data has been saved to 'nobroker_house_details.csv'")

                                               Title             Location  \
0  2 BHK Flat In Ramya Regent For Sale  In Indira...  near police station   

              Area                         EMI     Price  
0  917 sqftBuiltup  ₹1 Crore₹10,905 per sq.ft.  ₹1 Crore  
Data has been saved to 'nobroker_house_details.csv'


In [ ]:
#Q4

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://www.bewakoof.com/bestseller?sort=popular'


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}


response = requests.get(url, headers=headers)


soup = BeautifulSoup(response.content, 'html.parser')


product_containers = soup.find_all('div', class_='productCardBox', limit=10)


products = []


for container in product_containers:
    try:
        
        name = container.find('h3').text.strip()
        
        
        price = container.find('div', class_='discountedPriceText clr-p-black false').text.strip()
        
        
        image_url = container.find('img')['src']
        
        
        products.append({
            'Product Name': name,
            'Price': price,
            'Image URL': image_url
        })
    except AttributeError:
        continue

products_df = pd.DataFrame(products)
print(products_df)
products_df.to_csv('bewakoof_product_details.csv', index=False)
print("Data has been saved to 'bewakoof_product_details.csv'")


                     Product Name Price  \
0          bewakoof x tom & jerry  ₹499   
1                   bewakoof x dc  ₹499   
2             bewakoof x garfield  ₹549   
3  bewakoof x house of the dragon  ₹399   
4                       Bewakoof®  ₹349   
5                       Bewakoof®  ₹439   
6                       Bewakoof®  ₹399   
7                       Bewakoof®  ₹499   
8                 bewakoof x nasa  ₹399   
9               bewakoof x marvel  ₹599   

                                           Image URL  
0  https://images.bewakoof.com/t640/women-s-blue-...  
1  https://images.bewakoof.com/t640/men-s-black-a...  
2  https://images.bewakoof.com/t640/women-s-green...  
3  https://images.bewakoof.com/t640/men-s-black-h...  
4  https://images.bewakoof.com/t640/women-s-red-i...  
5  https://images.bewakoof.com/t640/women-s-brown...  
6  https://images.bewakoof.com/t640/women-s-pink-...  
7  https://images.bewakoof.com/t640/men-s-sun-kis...  
8  https://images.bewakoof.com/

In [5]:
#Q5

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://www.cnbc.com/world/?region=world'


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}


response = requests.get(url, headers=headers)


soup = BeautifulSoup(response.content, 'html.parser')


articles = soup.find_all('div', class_='LatestNews-container')


news_data = []


for article in articles:
    try:
        
        heading = article.find('a', class_='LatestNews-headline').text.strip()
        
        
        date = article.find('time', class_='LatestNews-timestamp').text.strip()
        
        
        news_link = article.find('a', class_='LatestNews-headline')['href']
        news_link = 'https://www.cnbc.com' + news_link
        news_data.append({
            'Heading': heading,
            'Date': date,
            'Link': news_link
        })
    except AttributeError:
        continue

news_df = pd.DataFrame(news_data)
print(news_df)
news_df.to_csv('cnbc_world_news.csv', index=False)
print("Data has been saved to 'cnbc_world_news.csv'")


                                              Heading             Date  \
0   How Americans are losing their life savings to...       39 Min Ago   
1   Good economic data sparked a stock rally. Here...      4 Hours Ago   
2   With Harris economic plan, U.S. family and chi...      4 Hours Ago   
3   'Couples Therapy' host Orna Guralnik doesn't t...      4 Hours Ago   
4   This 37-year-old brings in more than $100K per...      4 Hours Ago   
5   3 side hustles for teachers—one can pay as muc...      5 Hours Ago   
6   Big fast-casual restaurant chains are rapidly ...      5 Hours Ago   
7   To fix Starbucks, incoming CEO Niccol will hav...      5 Hours Ago   
8   Former Google researcher's company wants to gi...      5 Hours Ago   
9   5 money moves to consider ahead of the Fed's f...      5 Hours Ago   
10  Chinese companies are already investing in the...      5 Hours Ago   
11  AI is unlocking new uses for military drones, ...      5 Hours Ago   
12  Stablecoins are growing but not as

In [7]:
#Q6

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://www.keaipublishing.com/en/journals/artificial-intelligence-in-agriculture/most-downloaded-articles/'


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}


response = requests.get(url, headers=headers)


soup = BeautifulSoup(response.content, 'html.parser')


papers_data = []


articles = soup.find_all('div', class_='article-listing')



for article in articles:
    try:
        
        title = article.find('h2').text.strip()

        date = article.find('p', class_='article-date').text.strip()

        author = article.find('p', class_='article-authors').text.strip()
        
        
        papers_data.append({
            'Title': title,
            'Date': date,
            'Author': author
        })
    except AttributeError:
        continue

papers_df = pd.DataFrame(papers_data)
print(papers_df)
papers_df.to_csv('most_downloaded_articles.csv', index=False)
print("Data has been saved to 'most_downloaded_articles.csv'")


                                                Title            Date  \
0   Implementation of artificial intelligence in a...            2020   
1               Review of agricultural IoT technology            2022   
2   Automation and digitization of agriculture usi...            2021   
3   A comprehensive review on automation in agricu...       June 2019   
4   Applications of electronic nose (e-nose) and e...            2020   
5   Towards sustainable agriculture: Harnessing AI...       June 2024   
6             Fruit ripeness classification: A survey      March 2023   
7   Deep learning based computer vision approaches...            2022   
8   A review of imaging techniques for plant disea...            2020   
9   Transfer Learning for Multi-Crop Leaf Disease ...            2022   
10  Comparison of CNN-based deep learning architec...  September 2023   
11  DeepRice: A deep learning and deep feature bas...      March 2024   
12  Computer vision in smart agriculture and preci.